## **4. Combine an *n-gram* with Wav2Vec2**

In a final step, we want to wrap the *5-gram* into a `Wav2Vec2ProcessorWithLM` object to make the *5-gram* boosted decoding as seamless as shown in Section 1.
We start by downloading the currently "LM-less" processor of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv).

In [257]:
# !sudo apt-get install ffmpeg

In [1]:
#run inference on test dataset first example
import soundfile as sf
import torch
from IPython.display import Audio
import numpy as np

In [11]:
!pip install pydub


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [117]:
#get first 10 seconds of the audio
from pydub import AudioSegment
from pydub.utils import make_chunks

import math

# Load your MP3 file
audio = AudioSegment.from_mp3("data/recordings/bavametzia_2.mp3")

# Define the length of each chunk in milliseconds
chunk_length_ms = 15000  # 10 seconds * 1000 ms/sec
chunks = make_chunks(audio, chunk_length_ms) 
chunks = [chunk.set_frame_rate(16000).set_channels(1) for chunk in chunks]
chunks = [np.array(chunk.get_array_of_samples()) for chunk in chunks]
chunks = [chunk.astype(np.float32) / np.abs(chunk).max() for chunk in chunks]
# Calculate the number of chunks to split the file into
# num_chunks = math.ceil(len(audio) / chunk_length_ms)
# chunks = []
# Split the audio and save each chunk
# for i in range(num_chunks):
#     start_ms = i * chunk_length_ms
#     end_ms = min((i + 1) * chunk_length_ms, len(audio))
#     chunk = audio[start_ms:end_ms]
#     chunks.append(chunk)

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM, AutoProcessor

model = Wav2Vec2ForCTC.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew")
processor = AutoProcessor.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew")

Some weights of the model checkpoint at imvladikon/wav2vec2-xls-r-300m-hebrew were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at imvladikon/wav2vec2-xls-r-300m-hebrew and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probab

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [66]:
processor_with_lm = Wav2Vec2ProcessorWithLM.from_pretrained("/teamspace/studios/this_studio/FinalProject/models/KenLM-Wav2Vec2-Hebrew-Mesivta")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/FinalProject/models/KenLM-Wav2Vec2-Hebrew-Mesivta/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [146]:
sample = chunks[46]
inputs = processor(sample, sampling_rate=16000, return_tensors="pt")

In [147]:
sf.write("test.wav", sample, 16000)

In [148]:
import torch

with torch.no_grad():
  logits = model(**inputs).logits
# predicted_ids = torch.argmax(logits, dim=-1)


In [149]:
kenlm = processor_with_lm.batch_decode(logits.numpy()).text

In [150]:
predicted_ids = torch.argmax(logits, dim=-1)
word2vec = processor.batch_decode(predicted_ids)

In [151]:
kenlm

['זה אומר אני מצאתיה וזה אומר כולה שלי עלו יחד אומרת הגמרא קן זה לא שתי ספורים בעדים וכו מתי או']

In [152]:
word2vec

['זה אומר אני מצתיה אוזה אור כולשלילי ת עות הללו יחד אומרת ההגמרה כנ  הקן זה לא שתי סיפורים שבבעדים וכוח הנימצתיה או']